In [18]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

# Read the csv file
df = pd.read_csv('dataset.csv')

# Map the labels to binary values
df['label'] = df['label'].map({'NORMAL': 0, 'PNEUMONIA': 1})

# extract matrix
mat = df.values



x_train, x_val, y_train, y_val = train_test_split(mat[:,1:], mat[:,0], test_size=0.2, random_state=42)

In [19]:
from keras.applications.densenet import DenseNet121
from keras.layers import Dense, GlobalAveragePooling2D
from keras.models import Model
from keras.preprocessing.image import ImageDataGenerator

# Preprocess the data
x_train = (x_train - (np.min(x_train))) / (np.max(x_train) - (np.min(x_train)))
x_val = (x_val - (np.min(x_val))) / (np.max(x_val) - (np.min(x_val)))
x_train = x_train.astype('float32')
x_val = x_val.astype('float32')
x_train = np.repeat(x_train, 3, axis=-1)
x_val = np.repeat(x_val, 3, axis=-1)
x_train = x_train.reshape(-1, 100, 100, 3)
x_val = x_val.reshape(-1, 100, 100, 3)

In [20]:
for i in [x_train,y_train,x_val,y_val]:
    print(i.shape)

(4185, 100, 100, 3)
(4185,)
(1047, 100, 100, 3)
(1047,)


In [21]:
# Data augmentation
datagen = ImageDataGenerator(
        featurewise_center=False,  # set input mean to 0 over the dataset
        samplewise_center=False,  # set each sample mean to 0
        featurewise_std_normalization=False,  # divide inputs by std of the dataset
        samplewise_std_normalization=False,  # divide each input by its std
        zca_whitening=False,  # apply ZCA whitening
        rotation_range=10,  # randomly rotate images in the range (degrees, 0 to 180)
        zoom_range = 0.1, # Randomly zoom image 
        width_shift_range=0.1,  # randomly shift images horizontally (fraction of total width)
        height_shift_range=0.1,  # randomly shift images vertically (fraction of total height)
        horizontal_flip=True,  # randomly flip images
        vertical_flip=False)  # randomly flip images

datagen.fit(x_train)

In [22]:
# Load the DenseNet121 model
base_model = DenseNet121(weights='imagenet', include_top=False, input_shape=(100, 100, 3))

# Freeze the layers
for layer in base_model.layers:
    layer.trainable = False

# Add a new classifier layer
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
x = Dense(512, activation='relu')(x)
predictions = Dense(1, activation='sigmoid')(x)

# Create the final model
model = Model(inputs=base_model.input, outputs=predictions)

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the model on your dataset
model.fit_generator(datagen.flow(x_train, y_train, batch_size=32),
                    steps_per_epoch=len(x_train) / 32, epochs=10,
                    validation_data=(x_val, y_val))

Epoch 1/10


C:\Users\LEOFEN~1\AppData\Local\Temp/ipykernel_211064/2106348888.py:22: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  model.fit_generator(datagen.flow(x_train, y_train, batch_size=32),


130/130 [==============================] - 50s 354ms/step - loss: 0.2594 - accuracy: 0.9068 - val_loss: 0.1729 - val_accuracy: 0.9265
Epoch 2/10
130/130 [==============================] - 45s 348ms/step - loss: 0.1677 - accuracy: 0.9338 - val_loss: 0.1292 - val_accuracy: 0.9570
Epoch 3/10
130/130 [==============================] - 45s 343ms/step - loss: 0.1307 - accuracy: 0.9474 - val_loss: 0.1575 - val_accuracy: 0.9293
Epoch 4/10
130/130 [==============================] - 44s 337ms/step - loss: 0.1385 - accuracy: 0.9438 - val_loss: 0.1318 - val_accuracy: 0.9465
Epoch 5/10
130/130 [==============================] - 44s 335ms/step - loss: 0.1295 - accuracy: 0.9448 - val_loss: 0.1080 - val_accuracy: 0.9570
Epoch 6/10
130/130 [==============================] - 43s 333ms/step - loss: 0.1248 - accuracy: 0.9517 - val_loss: 0.2185 - val_accuracy: 0.9169
Epoch 7/10
130/130 [==============================] - 45s 345ms/step - loss: 0.1282 - accuracy: 0.9484 - val_loss: 0.1217 - val_accuracy: 0.9

In [23]:
# Evaluate the model on the validation set
scores = model.evaluate(x_val, y_val, verbose=0)

# Print the accuracy
print("Accuracy: %.2f%%" % (scores[1]*100))


Accuracy: 94.94%
